In [1]:
using Dates
using LinearAlgebra
using SparseArrays
using TensorOperations
using Plots

### Motivation

The following notebook works through three methods &mdash; iterative diagonalization, truncated iterative diagonalization, and density matrix renormalization group (DMRG) &mdash; to find the the ground state energy of a spin-1 Heisenberg chain with open boundary conditions and nearest neighbor interactions. The varying accuracy and convergence times for the three method are compared, illustrating the relative performances.


### 1D chain of Spin-1 particles - Classical Heisenberg Model

A 1D spin chain with open boundary conditions and nearest neighbor interactions for spin-1 particles with coupling strength J is illustrated below.

<img src="./img/spin-1%20chain.png" alt="spin-1 chain" width="50%" style="display: block; margin-left: auto; margin-right: auto;"/>

The model is described by the following Hamiltonian,

$$\mathcal{H} = J \sum_{i=1}^{N-1} \vec{S}_i \cdot \vec{S}_{i+1},$$

where $N$ is the number of spins and $\vec{S}_j = (S^x_j, S^y_j, S^z_j)$ is the vector of spin-1 operators at site $j$.

Introducing the usual raising and lowering operators $S^\pm = S^x \pm iS^y$ and considering the basis $\{ |s^z_j\rangle \} = \{ |1\rangle,|0\rangle,|-1\rangle \}$, such that

\begin{align*}
 S^+ |1\rangle &= 0 \qquad & S^+ |0\rangle &= |1\rangle \qquad & S^+ |-1\rangle &= |0\rangle \\
S^- |1\rangle &= |0\rangle\qquad &   S^- |0\rangle &= |-1\rangle \qquad & S^- |-1\rangle &= 0,
\end{align*}

the above Hamiltonian can be rewritten as

$$\mathcal{H} = J \sum_{j=1}^{N-1} \frac{1}{2} \left( S^+_j S^-_{j+1} + S^-_j S^+_{j+1} \right) +  S^z_j S^z_{j+1}.$$

In the $|s^z\rangle$ basis the operators $S^+, S^-,$ and $S^z$ have the folloing representation,

$$S^+ = \sqrt{2} \begin{pmatrix} 0 & 1 & 0 \\ 0 & 0 & 1 \\ 0 & 0 & 0 \end{pmatrix}, \quad S^- = \sqrt{2} \begin{pmatrix} 0 & 0 & 0 \\ 1 & 0 & 0 \\ 0 & 1 & 0 \end{pmatrix}, \quad S^z = \begin{pmatrix} 1 & 0 & 0 \\ 0 & 0 & 0 \\ 0 & 0 & -1 \end{pmatrix}.$$

In [2]:
# Defining the spin-1 operators

## Sz
Sz = zeros(3, 3)
Sz[1, 1] = 1
Sz[3, 3] = -1

## Sp
Sp = zeros(3, 3)
Sp[1, 2] = sqrt(2)
Sp[2, 3] = sqrt(2)

## Sm
Sm = zeros(3, 3)
Sm[2, 1] = sqrt(2)
Sm[3, 2] = sqrt(2)

## I
I = Matrix(Diagonal(ones(3)));

### Iterative diagonalization

This section details the calculation of the ground state energy for spin-1 Heisenberg chains using iterative diagonalization. This approach yields exact results, limited only by machine precision, as it avoids any truncation of the Hilbert space. However, the exponential growth of the Hilbert space dimension restricts its applicability to short chain lengths.

Connecting iterative diagonalization to DMRG, the following simulation begins with two spin-1 particles, one for each of the system (S) and environment (E) blocks. With each iteration two particles are added, one to each block. The following depicts a full iteration, with the superblock labeled SB. 

<br>

<img src="./img/ID_scheme.png" alt="ID scheme" width="50%" style="display: block; margin-left: auto; margin-right: auto;"/>


In [3]:
# Parameters
Nit = 3  # number of iterations

# Assume Sz, Sp, Sm, and I are defined globally before this code block

# Iterative diagonalization
start_time = now()
I_S = I
H_S = zeros(size(I_S))
Szborder_S = Sz
Spborder_S = Sp
Smborder_S = Sm
I_E = I
H_E = zeros(size(I_E))
Szborder_E = Sz
Spborder_E = Sp
Smborder_E = Sm
I_L = I
H_L = zeros(size(I_L))
Szborder_L = Sz
Spborder_L = Sp
Smborder_L = Sm
I_R = I
H_R = zeros(size(I_R))
Szborder_R = Sz
Spborder_R = Sp
Smborder_R = Sm

for k in 1:Nit
    if k > 1  # remaining iterations
        I_S = I_L
        H_S = H_L
        Szborder_S = Szborder_L
        Spborder_S = Spborder_L
        Smborder_S = Smborder_L

        I_E = I_R
        H_E = H_R
        Szborder_E = Szborder_R
        Spborder_E = Spborder_R
        Smborder_E = Smborder_R
    end

    ## L = S x l
    I_L = kron(I_S, I)
    H_L = kron(H_S, I) + kron(Szborder_S, Sz) + 0.5 * (kron(Spborder_S, Sm) + kron(Smborder_S, Sp))
    Szborder_L = kron(I_S, Sz)
    Spborder_L = kron(I_S, Sp)
    Smborder_L = kron(I_S, Sm)

    ## R = r x E
    I_R = kron(I, I_E)
    H_R = kron(I, H_E) + kron(Sz, Szborder_E) + 0.5 * (kron(Sp, Smborder_E) + kron(Sm, Spborder_E))
    Szborder_R = kron(Sz, I_E)
    Spborder_R = kron(Sp, I_E)
    Smborder_R = kron(Sm, I_E)

    ## SB = L x R
    H_SB = kron(H_L, I_R) + kron(I_L, H_R) + kron(Szborder_L, Szborder_R) + 0.5 * (kron(Spborder_L, Smborder_R) + kron(Smborder_L, Spborder_R))

    ## diagonalization
    eigenvalues = eigvals(Hermitian(H_SB))
    sort!(eigenvalues)
    En = eigenvalues

    ## outputs
    Nspins = 2 + 2 * k
    println("#spins = ", Nspins)
    Egs = En[1]
    println("GS energy = ", Egs)
    println("dim of SB = ", size(H_SB, 1))
    time_elapsed = Dates.value(now() - start_time) / 1000.0
    println("time of iteration (s) = ", time_elapsed)
    start_time = now()
    println("\n")
end

#spins = 4
GS energy = -4.6457513110645925
dim of SB = 81
time of iteration (s) = 0.915


#spins = 6
GS energy = -7.370274969424611
dim of SB = 729
time of iteration (s) = 0.115


#spins = 8
GS energy = -10.124637222358913
dim of SB = 6561
time of iteration (s) = 18.553


